In [25]:
import json
import surprise
import pandas as pd
from math import ceil
import datetime as dt

from app import db
from app.models.utils.timer import Timer
from app.helper import get_saved_model, save_model
from app.models.utils.ImplicitCF import python_random_split, DEFAULT_USER_COL, DEFAULT_ITEM_COL, DEFAULT_PREDICTION_COL


class Recommendation:
    
    def __init__(self, model_id='recommendation'):
        # Top k items to recommend
        self.TOP_K = 10
        self.model_id = model_id
        self.min_no_of_recommendations = 1
        self.min_prediction_score = 2.5
        self.usercol=DEFAULT_USER_COL
        self.itemcol=DEFAULT_ITEM_COL
        self.predcol=DEFAULT_PREDICTION_COL
        
        # Load datset
        self.dataset, self.rec_master_df = self.load_dataset()
        
    
    def fn(self, row):
        max_usage = row['cpuusage'] #, row['memoryusage'], row['diskusage'])
        return ceil(max_usage/10)    
    
    
    def load_dataset(self):
        filepath = r'./data/recommendation.csv'
        rec_dataset = pd.read_csv(filepath)#, nrows=10000)
        rec_dataset.rename(columns={clm: clm.lower() for clm in rec_dataset.columns}, inplace=True)
        rec_dataset['usage_level'] = rec_dataset.apply(lambda row: self.fn(row), axis=1)
        # Binarize the data (only keep high usages >= 7)
        # df = df[(df['usage_level'] < 5) & (df['usage_level'] > 7)]

        label_features = ['servicename', 'cloudprovider', 'pricingmodel', 'environment', 'usage_level']#, 'cpu_usage', 'memory_usage', 'disk_usage']
        # features_id = df.set_index(label_features).index.factorize()[0]+1
        # df.drop('features_id', axis=1, inplace=True, errors='ignore')
        # df.insert(loc=0, column='features_id', value=features_id)

        # recommendation_id = rec_dataset.set_index(['recommendation']).index.factorize()[0]+1
        # rec_dataset.drop('recommendation_id', axis=1, inplace=True, errors='ignore')
        # rec_dataset.insert(loc=1, column='recommendation_id', value=recommendation_id)
        # print(recommendation_id)
        
        # Get all the recommendation as a dataframe
        rec_master_df = list(db.df_fetch_data('recommendationmaster'))[0]
        return rec_dataset, rec_master_df


self = Recommendation()


Executing SELECT * FROM recommendationmaster ...


In [26]:
data['feature_usage'] = data.apply(lambda row: str(row['feature_id']) + '_' + str(row['usage_level']), axis=1)
feature_usage_id = data.set_index(['feature_usage']).index.factorize()[0]+1
data.drop(['feature_usage_id', 'rating'], axis=1, inplace=True, errors='ignore')
data.insert(loc=1, column='feature_usage_id', value=feature_usage_id)
data.insert(loc=2, column='rating', value=5)
data

AttributeError: 'DatasetAutoFolds' object has no attribute 'apply'

In [14]:
data, rec_master_df = self.dataset, self.rec_master_df

from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

# Define the reader
reader = Reader(rating_scale=(1, 5))

# Load the data into Surprise
data = Dataset.load_from_df(data[['feature_usage_id', 'rec_masterid', 'rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.25)


from surprise import SVD
from surprise import accuracy

# Initialize the SVD model
model = SVD()

# Train the model
model.fit(trainset)

# Make predictions
predictions = model.test(testset)

# Evaluate the model
accuracy.rmse(predictions)


RMSE: 0.0026


0.002629221332477199

In [15]:
predictions

[Prediction(uid=84, iid=3, r_ui=5.0, est=5, details={'was_impossible': False}),
 Prediction(uid=66, iid=7, r_ui=5.0, est=5, details={'was_impossible': False}),
 Prediction(uid=131, iid=0, r_ui=5.0, est=4.998692659531685, details={'was_impossible': False}),
 Prediction(uid=41, iid=6, r_ui=5.0, est=5, details={'was_impossible': False}),
 Prediction(uid=185, iid=7, r_ui=5.0, est=4.999203516156912, details={'was_impossible': False}),
 Prediction(uid=236, iid=0, r_ui=5.0, est=4.995000849967047, details={'was_impossible': False}),
 Prediction(uid=51, iid=0, r_ui=5.0, est=4.998676222441367, details={'was_impossible': False}),
 Prediction(uid=174, iid=1, r_ui=5.0, est=4.996368164663438, details={'was_impossible': False}),
 Prediction(uid=321, iid=2, r_ui=5.0, est=5, details={'was_impossible': False}),
 Prediction(uid=204, iid=2, r_ui=5.0, est=5, details={'was_impossible': False}),
 Prediction(uid=275, iid=0, r_ui=5.0, est=4.998011417374442, details={'was_impossible': False}),
 Prediction(uid=1

In [17]:
predictions = pd.DataFrame(predictions)
predictions = predictions.rename(
    index=str, columns={"uid": 'feature_usage_id', "iid": 'rec_masterid', "est": 'rating'}
)
predictions

,feature_usage_id,rec_masterid,r_ui,rating,details
0,84,3,5.0,5.000000,{'was_impossible': False}
1,66,7,5.0,5.000000,{'was_impossible': False}
2,131,0,5.0,4.998693,{'was_impossible': False}
3,41,6,5.0,5.000000,{'was_impossible': False}
4,185,7,5.0,4.999204,{'was_impossible': False}
...,...,...,...,...,...
4090,65,1,5.0,4.996796,{'was_impossible': False}
4091,165,0,5.0,5.000000,{'was_impossible': False}
4092,218,0,5.0,5.000000,{'was_impossible': False}
4093,110,1,5.0,4.998905,{'was_impossible': False}


In [23]:
self.dataset.loc[(self.dataset['feature_usage_id'] == 84) & ((self.dataset['usage_level'] == 6)) ]
# predictions.loc[predictions['feature_usage_id'] == 84]

,feature_id,feature_usage_id,rating,cpuusage,memoryusage,diskusage,rec_masterid,recommendation,usage_level,feature_usage
84,245,84,5,50,60,10,0,NaN,6,245_6
174,245,84,5,50,60,10,0,NaN,6,245_6
264,245,84,5,50,60,10,0,NaN,6,245_6
354,245,84,5,50,60,10,0,NaN,6,245_6
444,245,84,5,50,60,10,0,NaN,6,245_6
...,...,...,...,...,...,...,...,...,...,...
14484,245,84,5,11,60,50,3,Use Serverless options for Amazon Simple Stora...,6,245_6
14574,245,84,5,11,60,50,3,Use Serverless options for Amazon Simple Stora...,6,245_6
14664,245,84,5,11,60,50,3,Use Serverless options for Amazon Simple Stora...,6,245_6
14754,245,84,5,11,60,50,3,Use Serverless options for Amazon Simple Stora...,6,245_6


In [37]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Flatten, Concatenate, Dense, Input
from tensorflow.keras.models import Model

# Define the model
def build_ncf_model(num_users, num_items, embedding_size=50):
    # User and item input layers
    user_input = Input(shape=(1,))
    item_input = Input(shape=(1,))

    print(user_input)
    
    # Embedding layers
    user_embedding = Embedding(num_users, embedding_size)(user_input)
    item_embedding = Embedding(num_items, embedding_size)(item_input)
    
    # Flatten the embeddings
    user_embedding = Flatten()(user_embedding)
    item_embedding = Flatten()(item_embedding)
    
    # Concatenate user and item embeddings
    concatenated = Concatenate()([user_embedding, item_embedding])
    
    # Fully connected layers
    dense = Dense(128, activation='relu')(concatenated)
    output = Dense(1, activation='sigmoid')(dense)
    
    # Define the model
    model = Model(inputs=[user_input, item_input], outputs=output)
    return model

# Build the model
num_users = self.dataset['feature_id'].nunique()
num_items = self.dataset['rec_masterid'].nunique()
model = build_ncf_model(num_users, num_items)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

<KerasTensor shape=(None, 1), dtype=float32, sparse=False, name=keras_tensor_72>


In [36]:
# Prepare the data
user_ids = self.dataset['feature_id'].values
item_ids = self.dataset['rec_masterid'].values
ratings = self.dataset['usage_level'].values
# self.dataset['ratings'] = 5

# Train the model
model.fit([user_ids, item_ids], ratings, epochs=10, batch_size=64, validation_split=0.2)


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node functional_7_1/embedding_14_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\install\works\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\install\works\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\install\works\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "C:\install\works\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\install\works\anaconda3\Lib\asyncio\windows_events.py", line 322, in run_forever

  File "C:\install\works\anaconda3\Lib\asyncio\base_events.py", line 641, in run_forever

  File "C:\install\works\anaconda3\Lib\asyncio\base_events.py", line 1987, in _run_once

  File "C:\install\works\anaconda3\Lib\asyncio\events.py", line 8

In [27]:
# self.dataset.loc[self.dataset['feature_id'] == 286]
# self.dataset['rec_masterid'].max()

# self.dataset.loc[self.dataset.index == 286]
self.dataset

,feature_id,cpuusage,memoryusage,diskusage,rec_masterid,recommendation,usage_level,ratings
0,251,50,11,50,0,NaN,5,5
1,252,50,11,50,0,NaN,5,5
2,253,20,30,20,1,Go for lower configuration for Microsoft.Stora...,3,5
3,254,20,20,89,1,Go for lower configuration for Microsoft.Sql/s...,9,5
4,255,11,11,40,1,Go for lower configuration for Microsoft.Docum...,4,5
...,...,...,...,...,...,...,...,...
16375,246,40,89,40,6,Use Serverless options-Use reserved instance f...,9,5
16376,247,70,70,48,0,NaN,7,5
16377,248,60,79,98,0,NaN,10,5
16378,249,70,70,48,0,NaN,7,5
